<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class ContrastiveNetwork(nn.Module):
    def __init__(self, feature_dim):
        super(ContrastiveNetwork, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim)
        )

    def forward(self, x):
        return self.encoder(x)

def contrastive_loss(z1, z2, temperature=0.5):
    batch_size = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim_matrix = torch.exp(torch.mm(z, z.t()) / temperature)
    mask = torch.eye(sim_matrix.size(0), device=z.device).bool()
    sim_matrix = sim_matrix.masked_fill(mask, 0)
    sim_sum = sim_matrix.sum(dim=-1, keepdim=True)
    loss = -torch.log(sim_matrix / sim_sum).mean()
    return loss

# Example usage
feature_dim = 128
model = ContrastiveNetwork(feature_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    x1 = torch.randn(32, 784)  # Two augmentations of the same batch
    x2 = torch.randn(32, 784)
    optimizer.zero_grad()
    z1, z2 = model(x1), model(x2)
    loss = contrastive_loss(z1, z2)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')